In [92]:
import cv2
import os
import numpy as np
import pandas as pd

src_path = r"F:\Work\SOCAAutomation\Dataset\Converted"
dst_path = r"F:\Work\SOCAAutomation\Dataset\Images\SplitVideos"
excel = pd.read_excel(r"F:\Work\SOCAAutomation\Dataset\DatasetAnalysisReport.xlsx", sheet_name = 'Video Extraction')


# video_path = r"F:\Work\SOCAAutomation\Dataset\Converted\22_08_2024\AXIS 213 - 10.1.5.173 2024-08-22_10_40_00_000.mp4"
# output_folder = "frames_section"
output_folder = "Frames"
os.makedirs(output_folder, exist_ok=True)



In [77]:
excel['Start Time']

0     04;48
1     04;53
2     05;05
3     05;12
4     05;15
5     06;19
6     06;25
7     06;43
8     08;28
9     08;33
10    08;34
11    08;37
12    08;39
13    09;09
14    09;12
15    20;39
16    20;58
17    21;00
18    21;04
19    21;14
20    23;15
21    24;05
22    24;28
23    24;40
Name: Start Time, dtype: object

In [93]:
excel_group = pd.DataFrame()
excel_group['Start Time'] = excel.groupby(['Folder', 'Video File'])['Start Time'].apply(list)
excel_group['End Time'] = excel.groupby(['Folder', 'Video File'])['End Time'].apply(list)
excel_group.reset_index(inplace = True)
excel_group.head()

,Folder,Video File,Start Time,End Time
0,22_08_2024,AXIS 213 - 10.1.5.173 2024-08-22_10_40_00_000,"[04;48, 04;53, 05;05, 05;12, 05;15, 06;19, 06;...","[04;51, 04;54, 05;05, 05;14, 05;17, 06;21, 06;..."


In [96]:
fps = 30
for i in range(len(excel_group)):
    file_path = src_path + "/" + excel_group.loc[i, 'Folder'] + "/" + excel_group.loc[i, 'Video File'] + ".mp4"
    folder = dst_path + "/" + excel_group.loc[i, 'Video File'].split()[-1]
    cap = cv2.VideoCapture(file_path)
    segment = 1
    start_time_list = list(excel_group['Start Time'])[0]
    end_time_list = list(excel_group['End Time'])[0]
    for j in range(len(start_time_list)):
        
        start_time = start_time_list[j]
        end_time = end_time_list[j]
        
        print(f"Starting Segment {segment} => {start_time} to {end_time}")
        output_folder = folder +  "/" + str(segment)
        os.makedirs(output_folder, exist_ok=True)
    
        start_min = int(start_time.split(";")[0])
        start_sec = int(start_time.split(";")[1])
        start_sec = start_min * 60 + start_sec

        end_min = int(end_time.split(";")[0])
        end_sec = int(end_time.split(";")[1]) + 1
        end_sec = end_min * 60 + end_sec

        start_frame = int(start_sec * fps)
        end_frame   = int(end_sec * fps)

        frame_no = start_frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        print(f"Start Frame: {start_frame}, End Frame: {end_frame}")
        while frame_no <= end_frame:
            ret, frame = cap.read()
            if not ret:
                break

            filename = os.path.join(output_folder, f"frame_{frame_no:06d}.jpg")
            cv2.imwrite(filename, frame)

            frame_no += 1
        segment += 1

    cap.release()


Starting Segment 1 => 04;48 to 04;51
Start Frame: 8640, End Frame: 8760
Starting Segment 2 => 04;53 to 04;54
Start Frame: 8790, End Frame: 8850
Starting Segment 3 => 05;05 to 05;05
Start Frame: 9150, End Frame: 9180
Starting Segment 4 => 05;12 to 05;14
Start Frame: 9360, End Frame: 9450
Starting Segment 5 => 05;15 to 05;17
Start Frame: 9450, End Frame: 9540
Starting Segment 6 => 06;19 to 06;21
Start Frame: 11370, End Frame: 11460
Starting Segment 7 => 06;25 to 06;27
Start Frame: 11550, End Frame: 11640
Starting Segment 8 => 06;43 to 06;45
Start Frame: 12090, End Frame: 12180
Starting Segment 9 => 08;28 to 08;30
Start Frame: 15240, End Frame: 15330
Starting Segment 10 => 08;33 to 08;34
Start Frame: 15390, End Frame: 15450
Starting Segment 11 => 08;34 to 08;35
Start Frame: 15420, End Frame: 15480
Starting Segment 12 => 08;37 to 08;38
Start Frame: 15510, End Frame: 15570
Starting Segment 13 => 08;39 to 08;40
Start Frame: 15570, End Frame: 15630
Starting Segment 14 => 09;09 to 09;15
Start 

In [91]:
file_path

'22_08_2024/AXIS 213 - 10.1.5.173 2024-08-22_10_40_00_000.mp4'

In [87]:
for a, b in enumerate([['1', '2'], ['3', '4']]):
    print(a, b)

0 ['1', '2']
1 ['3', '4']


In [11]:
ret, frame = cap.read()

In [12]:
ret

False

In [14]:
cap.set(cv2.CAP_PROP_POS_MSEC, start_sec * 1000)

False